# Analog Circuit Emulator

This notebook includes conceptual and programmatic details for the emulator.

## Modified Nodal Analysis

This program is based on [MNA](https://en.wikipedia.org/wiki/Modified_nodal_analysis), a linear approximation of analog circuit behavior. It involves systematic calculation of voltages at nodes in the circuit using Kirchoff's laws. We begin with `Ohm's law`: 
$$
V=I\,R
$$
where $V$ is voltage (potential), $I$ is current, and $R$ is resistance. 

We use nodal form to set up the framework for dealing with many currents, voltages, and resistances distributed over a circuit. Note that conductance $G$ is the inverse of resistance ($1/R$). Conductance is also sometimes called 'admittance'. So we have: 
$$
I=V\,G
$$

It's important to know that circuit diagrams are referenced in the direction of positive flow by convention, thus the reagion of largest potential is the cathode, and the lowest potential is the anode. Of course, this is backward from the actual electron flow. Mathematically, this doesn't matter, it just sets up the sign convention for each expression. 

In this model, the voltage is fixed between two nodes. The highest potential is at the positive pole of the voltage source (a.k.a. emf) and the lowest potential is at the ground. For example if the voltage source is a battery, the ground is the anode. Apparently, the ground can be any reference point in the circuit, but is always given a zero voltage. 

MNA is a linear analysis where: 
$$
G\cdot x= I
$$

where $G$ is the `conductance matrix` that captures all of the constraints in the circuit (e.g., ohmic resistance). $x$ is a `vector of unknowns`, including voltages at each node and currents through voltage sources. $I$ is the `source vector`, including known voltage sources and currents. 

There are two important relations to keep in mind: 
1. Kirchoff's Current Law- states that the sum of currents at any node must equal 0. That is, the currents are signed. If $+1$ comes in $-1$ must leave.  This is similar to the mass balance constraint in metabolic flux analysis. 
2. Kirchoff's Voltage Law- states that within a closed loop, the total voltage sums to the voltage at the power source. In other words, the sum of voltage drops = the sum of voltage rises. 

## Simple Example

We'll start with a simple divider circuit with one voltage source 'V+' applying a 5.0 V potential from the positive pole 'V+' to the negative pole (ground; 'GND'). There is one resistance (1000$\Omega$), and arbitrary output device 'OUT', and a second (2000$\Omega$) resistance. All together: 

V+ -> Resistance -> OUT -> Resistance -> GND

Let V+ = $V_1$, OUT = $V_2$, and the current through the voltage source = $I_s$.

In nodal analysis we want to know: "What is the voltage at each node?"

KCL states that current can't accumulate, what comes into a node, must leave a node in equal proportion. KVL states that the potential in the circuit can't exceed the source (can't invent energy you don't have). 

The first equation of interest applies these laws from the first node to the second, i.e., V+ -> R1 -> OUT. In other words, we want to know the current from the source, through the first resistor to the OUT device; and we want to know what effect the resistance has on the voltage at the OUT node. Recall that $V\,G=I$. Through R1, the conductance is $\frac{1}{1000}(\Omega)$. So the total current is the difference in current across the resistor. 
$$
I_s= I_{s} - I_{OUT} = V_1\frac{1}{1000} - V_2\frac{1}{1000}
$$

KCL states that the total current at any node must be $0$, so, this gives us the first of three linear equations in the system. It describes the current through resistance R1 and the current through the voltage source. It is of the form:
$$
\frac{V_1-V_2}{1000} - I_s = 0
$$

When arranging the terms think about it like summing the nodes. If 'away from the node' is positive by convention, 'to the node' is negative. So to go from V+ to OUT, we sum the currents which are given by $+V_1\,\frac{1}{R1}$ and $-V_2\,\frac{1}{R1})$. In other words, the sum of the currents through the voltage source and through its connected resistor must be zero. Current in, current out. Now, in order to solve this in matrix form later, we need to be able to separate the unknowns, so we rearrange as follows: 

$$
V_1\,\frac{1}{1000} - V_2\,\frac{1}{1000} - (1)I_s = 0
$$

This will give us this nice vector of unknowns $x$: 

$$
x= \begin{bmatrix} 
V_1 \\ 
V_2 \\ 
I_s 
\end{bmatrix}
$$

Next, we formulate an equation for the current from OUT -> R2 -> GND. This is the second linear equation of the form: 
$$
\frac{V_2-V_1}{1000} + \frac{V_2 - 0}{2000} - (0)\,I_s = 0 
$$

The left term is the current through R1 (notice that is has the opposite sign of the expression in the first equation), the second term is the current through R2. Recall that the GND always has a potential of 0 V. The third term is a placeholder for the unknown current through the source, but is simply 0 here.

Again, to write this in matrix form for the whole system, each term can only have one variable, so we rearrange as follows: 
$$
-V_1\,\frac{1}{1000} + V_2\,(\frac{1}{1000} + \frac{1}{2000}) - (0)\,I_s = 0
$$

The final equation in the system describes the total voltage constraint on the system. It is the voltage from source to GND. 

$$
V_1 - (0)\,V_2 - (0)\,I_s = 5
$$

Now we can put the whole thing in matrix form to solve for our three unknowns: 

$$
\begin{bmatrix}
\phantom{-}0.001 & -0.001  & -1     \\
-0.001  & \phantom{-}0.0015 & \phantom{-}0     \\
\phantom{-}1     & 0        & 0
\end{bmatrix}
\begin{bmatrix}
V_1 \\
V_2 \\
I_s
\end{bmatrix}
=
\begin{bmatrix}
0 \\
0 \\
5
\end{bmatrix}
$$

## Adding Dynamics

MNA described above only returns the distribution of voltages among the nodes at steaty state. To emulate analog computers, we need to model dynamic processes such as integration using a combination of resistance and capacitation. To achieve this we'll add control flow to a new simulation function that enables calculating the MNA distribution for timesteps of a designated size. 

## The RC Circuit Solves a Special Case

In this example, we'll build a simple analog [RC circuit](https://en.wikipedia.org/wiki/RC_circuit) using our emulator program. The components include a voltage source, resistance, oscilloscope, and capacitor.

---
The setup. This program emulates the following circuit: 

$V_{in}(1 V step)$ -> R -> $V_{OUT}$ (Oscilloscope) -> C (Capacitor) -> GND

This setup will enable analog integration of a first order (linear) ordinary differential equation (ODE). Note that the 'step' in the voltage source just means that the source goes instantaneously from 0V to 1.0V when we turn it on. In other words, this idealized source does not produce any other voltage value (and is also noiseless).  

---
Some relevant physical relations: 

1. Ohm's law for resistance between two nodes: 

$$
I_R= \frac{(V_{in}-V_{OUT})}{R}
$$

2. Capacitor current-voltage relation:

$$
I_C= C\,\frac{dV_{OUT}}{dt}
$$

From KCL, the sum of the currents must equal zero, so: 

$$
\frac{(V_{in}-V_{OUT})}{R} = C\,\frac{dV_{OUT}}{dt}
$$

This rearranges to: 

$$
\frac{dV_{out}}{dt}= \frac{1}{R\,C}\,(V_{in}-V_{OUT})
$$

This is a first order, linear ODE of the form: 

$$
\frac{dy}{dt}= \frac{1}{R\,C}(x(t)-y(t))
$$
This ODE computes the exponentially weighted difference between $V_{in}=1$ (step input) and the output voltage at each timestep, recorded smoothly using using an emulated [oscilloscope](https://en.wikipedia.org/wiki/Oscilloscope). 

In other words for $V_{in}=1$ (the RC chargning curve for a 1V step input), the solution is: 

$$
V_{OUT}(t)=1-e^{\frac{-t}{RC}}
$$

## RC Circuit as a solver

The general form of a first order linear ODE is: 

$$
\frac{dy}{dt}= a\cdot\,x(t)- b\cdot\,y(t)
$$

So the RC circuit described above can compute the solution to any equation for which $a=b$. In this case: 

$$
a=b=\frac{1}{R\,C}
$$

From above:

$$
\frac{dy}{dt} = \frac{1}{RC} \, (x(t) - y(t))
$$

the solution for $y(t)$ is:

$$
y(t)= y(0)\, e^{\frac{-t}{RC}} + \int_0^t \frac{1}{RT}\,e^{-\frac{t-|tau}{RC}}\,x(\tau)\,d\tau
$$

where $\tau$ is a dummy variable for integration. In order to fit the solution to other types of problems, for example, to solve the velocity at time (t) from the measurements of an accelerometer, we would need to change the resistance and/or capacitor properties in the circuit to make the circuit's rate of voltage accumulation in the capacitor `analogous` to the problem at hand.

Here's how we would do this... You start with modeling the problem of the general form: 

$$
\frac{dy}{dt}= a(x-y)
$$

Then, 

$$
R\,C= \frac{1}{a}
$$

The product of RC has time units. It is known as the [RC time constant](https://en.wikipedia.org/wiki/RC_time_constant) ($\tau$; not to be confused with a dummy variable as $\tau$ is used in the integral above). 

So if we have an empirical measurement of the time constant $a$, we simply choose $R\,C$ such that the relation described above holds. 

For example if $a=100\,s^{-1}$, then $R\,C= \frac{1}{100}= 0.01\,s$ which can be achieved via either of the following arrangements: 
* $R = 10 \, \text{k}\Omega$, $C = 1 \, \mu\text{F}$
* $R = 1 \, \text{k}\Omega$, $C = 10 \, \mu\text{F}$

In this case, the circuit behaves as a parameter matched solver. 

## Bio-Note
If a biological system exhibits something analogous to self scaling $R\,C$, then that system can solve any linear ODE of the form: 

$$
\frac{dy}{dt}= \frac{1}{RC}(x(t)-y(t))
$$

within the limits of the combinations of $R\,C$ available to the biological system of course. Additionally, the system would need a way to 'set' the initial conditions in the form of an input potential (the voltage input in this case). This leads us to the integrator behavior of this circuit design. 

## RC Circuit as an Integrator

Suppose we have a sensor generates an output voltage that is proportional to acceleration $a(t)$ over time. The voltage output of the sensor is: 

$$
V_\text{sensor}(t)= k \cdot a(t)
$$

Where $k$ is some constant that relates the voltage to acceleration through the sensor. 

The velocity is related to the acceleration through,

$$
\frac{dv}{dt}= a(t)
$$

and,

$$
v(t)=\int_0^t a(t)\, dt
$$

If we set up the RC circuit so that is takes the accelerometer voltage as the input source, it would look like:

$V_{sensor}$ -> R -> OUT -> C -> GND

Recall that:

$$
\frac{dV_{out}}{dt}= \frac{1}{R\,C}\,(V_{in}-V_{OUT})
$$

In this case, the voltage from the accelerometer is the $V_{in}$ source. The $V_{OUT}$ will be the voltage read at the oscilloscope, this voltage is designated as being analogous to the velocity. So, 

$$
\frac{dV_{out}}{dt}= \frac{1}{R\,C}\,(V_{in}-V_{OUT}) \approx \frac{dv}{dt}= \frac{1}{RC} \, (k\cdot\,a(t) - v(t)) 
$$

This simple setup forms a 'leaky' integrator, because the integrated data leaks away through the capacitor. Thus, it is capable of approximate integration, but only over a narrow bandwidth and it will be sensitive to the balance between the input voltage dynamic range and the RC time constant. To get volecity units in the integration, we choose $RC=k$. 

## Adding an op-amp circuit
The RC circuit can solve a family of linear ODE's of the specific form we discussed above. The addition of an operational amplifier (op-amp) circuit will enable simulation of ANY first order linear ODE of the form: 

$$
\frac{dy}{dt}= a\cdot\,x(t)- b\cdot\,y(t)
$$

# componenents.py

This program defines class objects for different types of circuit components such as a voltage source (emf), resistor, etc. 